Projeto Final:\
\
Guia para Legend of Zelda: Breath of the Wild

In [12]:
from plyer import notification
from datetime import datetime
import pandas as pd
import requests
import sqlite3

In [13]:
def alerta(nivel, base, etapa):

    hoje = datetime.now()
    hoje = hoje.strftime("%d/%m/%Y %H:%M:%S")

    message = f'Falha no carregamento da base {base} na etapa {etapa}.\n{hoje}'
    
    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    
    notification.notify(
        title = title,
        message = message,
        app_name = 'alerta',
        timeout = 10)

Banco de Dados

In [14]:
def retrieve_table():
    conn = sqlite3.connect('zelda.db')
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()
    return schema

def save_db(df,table_name):
    conn = sqlite3.connect('zelda.db')
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()
    return True

def load_db(table_name):
    conn = sqlite3.connect('zelda.db')
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)

    conn.close()
    return df

Extração

In [15]:
url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
response = requests.get(url)

if response.status_code == 200:
    data_json = response.json()
    data_json = data_json['data']
else:
    print(f'Erro {response.status_code}')

compendium_df = pd.DataFrame(data_json)

In [16]:
id          = [id for id in compendium_df['id']]
name        = [name for name in compendium_df['name']]
locations   = [loc for loc in compendium_df['common_locations']]
category    = [cat for cat in compendium_df['category']]
drops       = [drop for drop in compendium_df['drops']]

item_df = pd.DataFrame({
    'id': id,
    'category': category,
    'locations': locations,
    'name': name,
    'drops': drops})

In [17]:
id          = [id for id in compendium_df['id']]
name        = [name for name in compendium_df['name']]
locations   = [loc for loc in compendium_df['common_locations']]
category    = [cat for cat in compendium_df['category']]
effect      = [fx for fx in compendium_df['cooking_effect']]
heal        = [heal for heal in compendium_df['hearts_recovered']]

food_df = pd.DataFrame({
    'id': id,
    'category': category,
    'locations': locations,
    'name': name,
    'effect': effect,
    'heal': heal})

Tratamento

In [18]:
item_df = item_df.explode('locations', ignore_index=True)
item_df = item_df.explode('drops', ignore_index=True)
item_df = item_df.dropna().sort_values(by='id', ascending=True)
food_df = food_df.explode('locations', ignore_index=True)
food_df = food_df.dropna().sort_values(by='id', ascending=True)
compendium_df = compendium_df.fillna('')
columns_drop = ['dlc', 'image', 'edible']
compendium_df = compendium_df.drop(columns=columns_drop, axis=1)
compendium_df = compendium_df.explode('drops', ignore_index=True)
compendium_df = compendium_df.explode('common_locations', ignore_index=True)
columns_astype = ['properties']
compendium_df[columns_astype] = compendium_df[columns_astype].apply(lambda i: i.astype('str'))

Load

In [19]:
save_db(item_df, 'item')
save_db(food_df, 'food')
save_db(compendium_df, 'compendium')
retrieve_table()

,name
0,item
1,food
2,compendium


In [20]:
load_db('item')

,id,category,locations,name,drops
0,9,creatures,Hyrule Ridge,bushy-tailed squirrel,acorn
1,9,creatures,Hyrule Field,bushy-tailed squirrel,acorn
2,10,creatures,Hyrule Field,woodland boar,raw meat
3,10,creatures,Lanayru Great Spring,woodland boar,raw meat
4,11,creatures,Deep Akkala,red-tusked boar,raw prime meat
...,...,...,...,...,...
432,388,treasure,Greater Hyrule,rare ore deposit,amber
433,388,treasure,Greater Hyrule,rare ore deposit,topaz
434,388,treasure,Greater Hyrule,rare ore deposit,flint
435,389,treasure,Greater Hyrule,luminous ore deposit,flint


In [21]:
load_db('food')

,id,category,locations,name,effect,heal
0,48,creatures,Hyrule Field,hyrule bass,,1.0
1,48,creatures,West Necluda,hyrule bass,,1.0
2,49,creatures,Akkala Highlands,hearty bass,extra hearts,2.0
3,49,creatures,West Necluda,hearty bass,extra hearts,2.0
4,50,creatures,Hyrule Field,staminoka bass,stamina recovery,1.0
...,...,...,...,...,...,...
134,198,materials,Lanayru Great Spring,blue nightshade,stealth up,0.0
135,199,materials,West Necluda,silent princess,stealth up,0.0
136,199,materials,Hyrule Ridge,silent princess,stealth up,0.0
137,200,materials,Tabantha Frontier,courser bee honey,stamina recovery,2.0


In [22]:
load_db('compendium')

,category,common_locations,description,drops,id,name,cooking_effect,hearts_recovered,properties
0,treasure,Greater Hyrule,This deposit contains a good deal of ore. Brea...,ruby,387,ore deposit,,,
1,treasure,Greater Hyrule,This deposit contains a good deal of ore. Brea...,diamond,387,ore deposit,,,
2,treasure,Greater Hyrule,This deposit contains a good deal of ore. Brea...,amber,387,ore deposit,,,
3,treasure,Greater Hyrule,This deposit contains a good deal of ore. Brea...,sapphire,387,ore deposit,,,
4,treasure,Greater Hyrule,This deposit contains a good deal of ore. Brea...,topaz,387,ore deposit,,,
...,...,...,...,...,...,...,...,...,...
1024,creatures,Hebra Mountains,This particular breed of grassland fox makes i...,raw prime meat,20,snowcoat fox,,,
1025,creatures,Tabantha Frontier,This particular breed of grassland fox makes i...,raw prime meat,20,snowcoat fox,,,
1026,creatures,Hebra Mountains,This particular breed of grassland fox makes i...,raw gourmet meat,20,snowcoat fox,,,
1027,creatures,Tabantha Frontier,This particular breed of grassland fox makes i...,raw gourmet meat,20,snowcoat fox,,,
